In [8]:
import numpy as np

def simulate_collisions(n=10000, d=1.0, xi_cut=4.8, check_conservation=False, save_to_file=False):
    """
    Функция для симуляции столкновений частиц с проверкой сохранения импульса и энергии.

    Параметры:
    n : int
        Количество частиц.
    d : float
        Прицельное расстояние.
    xi_cut : float
        Параметр для нормировки скоростей.
    check_conservation : bool
        Проверять ли сохранение импульса и энергии.
    save_to_file : bool
        Сохранять ли данные в файл.
    """


    # 1. Создание случайного массива параметров
    a = np.random.rand(8, n)
    
    # 2. Нормировка параметров
    # Скорости
    a[:6] = 2 * xi_cut * a[:6] - xi_cut
    
    # Прицельное расстояние [0, d]
    a[6] = d * a[6]
    
    # Азимутальный угол [0, 2π]
    a[7] = 2 * np.pi * a[7]
    
    # 3. Относительные скорости
    xi = a[:3].T  # (n, 3)
    xi1 = a[3:6].T
    g = xi1 - xi  # (n, 3)
    
    # 4. Углы отклонения θ (формула 1.14)
    b = a[6]
    theta = np.where(b <= d, 2 * np.arccos(b/d), 0.0)
    
    # 5. Преобразование относительных скоростей
    g_new = np.zeros_like(g)
    for i in range(n):
        g_i = g[i]
        eps = a[7,i]
        theta_i = theta[i]
        
        g_norm = np.linalg.norm(g_i)
        if g_norm < 1e-10:  # Нулевая относительная скорость
            continue
            
        # Случай 1.12a (общий случай)
        if np.abs(g_i[0]) + np.abs(g_i[1]) > 1e-10:
            g_xy = np.sqrt(g_i[0]**2 + g_i[1]**2)
            cos_eps = np.cos(eps)
            sin_eps = np.sin(eps)
            
            cos_theta = np.cos(theta_i)
            sin_theta = np.sin(theta_i)
            
            # Формулы 1.12a
            gx_new = (g_i[0]*cos_theta - (g_i[0]*g_i[2]/g_xy)*cos_eps*sin_theta 
                     + (g_i[1]/g_xy)*g_norm*sin_eps*sin_theta)
            gy_new = (g_i[1]*cos_theta - (g_i[1]*g_i[2]/g_xy)*cos_eps*sin_theta 
                     - (g_i[0]/g_xy)*g_norm*sin_eps*sin_theta)
            gz_new = g_i[2]*cos_theta + g_xy*cos_eps*sin_theta
            
            g_new[i] = [gx_new, gy_new, gz_new]
            
        # Случай 1.12b (g вдоль z)
        else:
            g_new[i,0] = g_norm * np.sin(theta_i) * np.sin(eps)
            g_new[i,1] = g_norm * np.sin(theta_i) * np.cos(eps)
            g_new[i,2] = g_norm * np.cos(theta_i)
    
    # 6. Скорости после столкновения (формула 1.8)
    xi_prime = 0.5*(xi + xi1) - 0.5*g_new
    xi1_prime = 0.5*(xi + xi1) + 0.5*g_new
    
    if check_conservation:
        # 7. Проверки сохранения
        # Сохранение относительной скорости
        g_prime = xi1_prime - xi_prime
        g_norm = np.linalg.norm(g, axis=1) # начальная относительная скорость
        g_prime_norm = np.linalg.norm(g_prime, axis=1) # конечная относительная скорость
        
        # Сохранение энергии
        energy_before = np.sum(xi**2 + xi1**2, axis=1)
        energy_after = np.sum(xi_prime**2 + xi1_prime**2, axis=1)
        
        # Сохранение импульса
        momentum_before = xi + xi1
        momentum_after = xi_prime + xi1_prime
        
        print("Проверки сохранения:")
        print(f"Относительная скорость: {np.max(np.abs(g_norm - g_prime_norm)):.2e}")
        print(f"Энергия: {np.max(np.abs(energy_before - energy_after)):.2e}")
        print(f"Импульс: {np.max(np.abs(momentum_before - momentum_after)):.2e}")

    if save_to_file:
        # Сохранение массивов в csv файл
        np.savetxt("xi_prime.csv", xi_prime, delimiter=",")
        np.savetxt("xi1_prime.csv", xi1_prime, delimiter=",")
        print("Данные сохранены в файлы xi_prime.csv и xi1_prime.csv")

    return xi_prime, xi1_prime

# Запуск симуляции
xi_prime, xi1_prime = simulate_collisions(check_conservation=True, save_to_file=True)

Проверки сохранения:
Относительная скорость: 3.55e-15
Энергия: 5.68e-14
Импульс: 1.78e-15
Данные сохранены в файлы xi_prime.csv и xi1_prime.csv
